In [3]:
import torch
import gc
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, BigBirdConfig

# [필수] 기존에 남아있는 쓰레기 메모리 정리
gc.collect()
torch.cuda.empty_cache()

# 1. 모델 및 토크나이저 설정
MODEL_NAME = "monologg/kobigbird-bert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config = BigBirdConfig.from_pretrained(MODEL_NAME)
config.num_labels = 5
# BigBird의 핵심인 sparse attention 사용 (original_full 금지)
config.attention_type = "block_sparse" 

# 모델 로드 시 low_cpu_mem_usage 옵션 추가
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    config=config,
    low_cpu_mem_usage=True
).to(device)

# [수정] 메모리 절약을 위한 Gradient Checkpointing 활성화
model.gradient_checkpointing_enable()

# 2. 전처리 함수 (길이를 256으로 줄여 메모리 확보)
def preprocess_function(examples):
    return tokenizer(examples["conversation"], truncation=True, max_length=256) 

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

# 3. 학습 인자 설정 (메모리 최적화 극대화)
training_args = TrainingArguments(
    output_dir="./results_bigbird",
    learning_rate=2e-5,
    per_device_train_batch_size=1,      # 최소 배치 사이즈
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,     # 1 * 16 = 실제 배치 16 효과
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,                          # Mixed Precision 필수
    optim="adamw_torch",                # 메모리 효율적인 옵티마이저
    gradient_checkpointing=True,        # 메모리 절약 핵심 설정
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer), 
    compute_metrics=compute_metrics
)

print("\n🦅 최적화 모드로 KoBigBird 학습 시작!")
trainer.train()

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BigBirdForSequenceClassification LOAD REPORT from: monologg/kobigbird-bert-base
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
classifier.out_proj.bias                   | MISSING    | 
classifier.out_proj.weight                 | MISSING    | 
classifier.dense.weight                    | MISSING    | 
classifier.dense.bias                      | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if 

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.58 GiB of which 7.62 MiB is free. Process 126 has 0 bytes memory in use. Process 646 has 0 bytes memory in use. Process 793 has 0 bytes memory in use. Process 1221 has 0 bytes memory in use. Including non-PyTorch memory, this process has 0 bytes memory in use. Of the allocated memory 324.38 MiB is allocated by PyTorch, and 25.62 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)